In [12]:
from datetime import datetime
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows', 1000)
import matplotlib.pyplot as plt

today = str(datetime.now().date())
plots_folder = f"./plots/{today}/iss"
os.system(f"mkdir -p {plots_folder}")

0

In [ ]:
from benfordslaw import benfordslaw

def bflaw_test(df: pd.DataFrame,
               cat_name: str = "all",
               alpha: float = 0.05,
               pos: int = 1,
              ):

    bl = benfordslaw(alpha=alpha,
                 pos=pos,
                 verbose=0,
                 method="chi2")

    categories = ["no_vax", "1_dose", "2_dosi_lt_cut", "2_dosi_gt_cut", "booster"]
    
    X = []
    for c in categories:
        if cat_name != "all" and c != cat_name:
            continue
        v = df[df.fascia_eta != "totale"][c].tolist()
        X += v
    
    X = np.array(X)
    
    # We do this because we are dealing with counts (only integers)
    if pos == 2:
        X = X[X > 9]
        
    
    print(f"Length X: {len(X)}")

    results = bl.fit(X)
    digits = np.array(results["percentage_emp"][:,0])
    obs = np.array(results["percentage_emp"][:,1])
    exp = np.array(bl.leading_digits)
    chi2 = np.round(results["t"],2)
    
    plt.figure(figsize=(10,5),dpi=90)
    plt.bar(digits, obs, width=0.5, label="Observed")
    plt.plot(digits, exp, "-o", color="r", markersize=9, label="Expected Benford")
    plt.xticks(digits)
    plt.grid()
    p_value = np.round(results["P"],2)
    
    if p_value <= bl.alpha:
        anomaly_str = " Anomaly detected!"
    else:
        anomaly_str = ""
        
    title = f"Benford's law test ISS data,"
    if cat_name != "all":
        title += f" ({cat_name})"
    title += f" position={pos}, chi2={chi2}, P={p_value} {anomaly_str}"
    title = title.replace("_"," ").replace('gt','>').replace('lt', '<')
    plt.title(title, fontsize=16)
    plt.yticks(fontsize=12)
    plt.xticks(fontsize=12)
    plt.legend(fontsize=12)
    plt.ylabel("Observed frequency [%]", fontsize=14)
    plt.savefig(f"{plots_folder}/bflawtest_pos{pos}_{cat_name}.png", bbox_inches="tight")
    return X

### Get data

In [ ]:
df_ti = pd.read_csv("./data/rapporti_ISS - TI.csv")
df_decessi = pd.read_csv("./data/rapporti_ISS - Decessi.csv")
df_ricoveri = pd.read_csv("./data/rapporti_ISS - Ricoveri.csv")
df_contagi = pd.read_csv("./data/rapporti_ISS - Contagi.csv")

In [ ]:
df_dict = {
    "contagi": df_contagi,
    "ricoveri": df_ricoveri,
    "terapie_intensive": df_ti, 
    "decessi": df_decessi
}

for d in df_dict.items():
    d[1]["split"] = d[0]

### Create single dataframe

In [ ]:
df_iss = pd.concat(df_dict.values()).reset_index(drop=True)

In [ ]:
_ = bflaw_test(df_iss, pos=1)

In [ ]:
_ = bflaw_test(df_iss, pos=2)